In [51]:
import pandas as pd

# Load CSV
df = pd.read_csv(r'C:\Users\clint\Desktop\Geocoding_Task\Yelp_Lookup\10.csv')

# Filter based on combined condition
enhanced_combined_condition = (
    (df['Scraped_phone_match_rate'] == True) |
    (df['Scraped_zipcode_to_label_match_rate'] == '6/6 successful match') |
    (df['Scraped_zipcode_to_label_match_rate'] == '7/6 successful match') |
    (df['Yelp_phone_match_rate'] == True)
)

filtered_df = df[~enhanced_combined_condition]
filtered_df

,clean_line1,clean_line2,line3,city,zip_code,label,phone,year,major_city,state,...,City_scraped_matches_row_ids_parsed,Exit_scraped_matches_row_ids_parsed,Road_scraped_matches_row_ids_parsed,Label_scraped_matches_row_ids_parsed,Chain_scraped_matches_row_ids_parsed,Scraped_zipcode_to_label_match_rate,Scraped_phone_match_rate,Phone_Yelp_matches_row_ids,Yelp_phone_match_rate,place_identifier(year)
288,"Washington , 84780 Freeway Chevron",435-673-3675 I-15 Exit 10,S <U+25A1>,Washington,84780,Freeway Chevron,435-673-3675,2006,Washington,UT,...,[],"[276, 287, 735, 746]","[16, 125, 156, 163, 168, 184, 205, 222, 229, 2...","[12, 24, 25, 28, 88, 100, 104, 119, 123, 129, ...","[12, 24, 25, 28, 88, 100, 104, 119, 123, 129, ...",2/6 successful match,False,NaN,False,49
289,Freeway Chevron,"435-673-3675 Washington , 84780 I-15 Exit 10",S D,NaN,84780,NaN,435-673-3675,2007,Washington,UT,...,[],"[276, 287, 735, 746]","[16, 125, 156, 163, 168, 184, 205, 222, 229, 2...",[],[],2/6 successful match,False,NaN,False,49
290,"Washington , 84780 Freeway Chevron",435-673-3675 I-15 Exit 10,24 S <U+2610><U+2610>,Washington,84780,Freeway Chevron,435-673-3675,2008,Washington,UT,...,[],"[276, 287, 735, 746]","[16, 125, 156, 163, 168, 184, 205, 222, 229, 2...","[12, 24, 25, 28, 88, 100, 104, 119, 123, 129, ...","[12, 24, 25, 28, 88, 100, 104, 119, 123, 129, ...",2/6 successful match,False,NaN,False,49
291,Washington ( 84780 ) Freeway Chevron,435-673-3675 I-15 Exit 10,24 S,Washington,84780,Freeway Chevron,435-673-3675,2014,Washington,UT,...,[],"[276, 287, 735, 746]","[16, 125, 156, 163, 168, 184, 205, 222, 229, 2...","[12, 24, 25, 28, 88, 100, 104, 119, 123, 129, ...","[12, 24, 25, 28, 88, 100, 104, 119, 123, 129, ...",2/6 successful match,False,NaN,False,49
292,Washington ( 84780 ) Freeway Chevron,435-673-3675 I-15 Exit 10,24 S <U+25A1>,Washington,84780,Freeway Chevron,435-673-3675,2015,Washington,UT,...,[],"[276, 287, 735, 746]","[16, 125, 156, 163, 168, 184, 205, 222, 229, 2...","[12, 24, 25, 28, 88, 100, 104, 119, 123, 129, ...","[12, 24, 25, 28, 88, 100, 104, 119, 123, 129, ...",2/6 successful match,False,NaN,False,49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,"Lakeport , 95453 Westgate Petroleum ( Alliance )",707-263-6281 1785 S Main ( 1/4 mi N of CA 29 &...,"175 ) DO 000 """,Lakeport,95453,Westgate Petroleum ( Alliance ),707-263-6281,2006,Lakeport,CA,...,"[272, 332, 731, 789]",[],"[2, 13, 19, 20, 24, 29, 37, 43, 47, 55, 56, 59...","[176, 177, 325, 332, 467, 483, 484, 647, 648, ...",[],5/6 successful match,False,NaN,False,486
2267,"Lakeport , 95453 Westgate Petroleum ( Alliance )",707-263-6281 1785 S Main ( 1/4 mi N of CA 29 &...,) <U+2610> 000 000,Lakeport,95453,Westgate Petroleum ( Alliance ),707-263-6281,2007,Lakeport,CA,...,"[272, 332, 731, 789]",[],"[2, 13, 19, 20, 24, 29, 37, 43, 47, 55, 56, 59...","[176, 177, 325, 332, 467, 483, 484, 647, 648, ...",[],5/6 successful match,False,NaN,False,486
2268,"Lakeport , 95453 Westgate Petroleum ( Alliance...",2 707-263-6281 1785 S Main ( 1/4 mi N of CA 29 &,193 <U+25A1> <U+25A1> <U+25A1>,Lakeport,95453,Westgate Petroleum ( Alliance ) 193,707-263-6281,2008,Lakeport,CA,...,"[272, 332, 731, 789]",[],"[2, 13, 19, 20, 24, 29, 37, 43, 47, 55, 56, 59...","[176, 177, 325, 332, 467, 483, 484, 647, 648, ...",[],5/6 successful match,False,NaN,False,486
2308,"Dunnigan , 95937 United Petroleum",530-724-3378 I-5 Exit 554 ( CR 8 W ),NaN,Dunnigan,95937,United Petroleum,530-724-3378,2006,Dunnigan,CA,...,"[250, 713]","[250, 713]","[105, 114, 138, 141, 152, 153, 154, 155, 157, ...","[176, 177, 318, 319, 325, 332, 467, 482, 483, ...",[],5/6 successful match,False,NaN,False,498


In [52]:
# Construct Google search queries using combined scraped fields
import urllib.parse

def build_google_search_url(row):
    # Combine relevant fields, handling missing values
    parts = [
        str(row.get('label', '')) or '',
        str(row.get('city', '')) or '',
        str(row.get('zip_code', '')) or '',
        str(row.get('phone', '')) or '',
        str(row.get('state', '')) or '',
        str(row.get('chain', '')) or ''
    ]
    # Remove empty parts and join with spaces
    query = ' '.join([p for p in parts if p.strip()])
    # Encode for URL
    encoded_query = urllib.parse.quote_plus(query)
    return f'https://www.google.com/search?q={encoded_query}'

# Apply to filtered DataFrame
filtered_df.loc[:, 'Google_Search_URL'] = filtered_df.apply(build_google_search_url, axis=1)

# Show a sample of the new column
filtered_df[[ 'Google_Search_URL']].head()


C:\Users\clint\AppData\Local\Temp\ipykernel_21260\558713760.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[:, 'Google_Search_URL'] = filtered_df.apply(build_google_search_url, axis=1)


,Google_Search_URL
288,https://www.google.com/search?q=Freeway+Chevro...
289,https://www.google.com/search?q=nan+nan+84780+...
290,https://www.google.com/search?q=Freeway+Chevro...
291,https://www.google.com/search?q=Freeway+Chevro...
292,https://www.google.com/search?q=Freeway+Chevro...


In [53]:
import urllib.parse
import webbrowser
import time

# Add DuckDuckGo and Bing search URLs for each row
def build_duckduckgo_search_url(row):
    parts = [
        str(row.get('label', '')) or '',
        str(row.get('city', '')) or '',
        str(row.get('zip_code', '')) or '',
        str(row.get('phone', '')) or '',
        str(row.get('state', '')) or '',
        str(row.get('chain', '')) or ''
    ]
    query = ' '.join([p for p in parts if p.strip()])
    encoded_query = urllib.parse.quote_plus(query)
    return f'https://duckduckgo.com/?q={encoded_query}'

def build_bing_search_url(row):
    parts = [
        str(row.get('label', '')) or '',
        str(row.get('city', '')) or '',
        str(row.get('zip_code', '')) or '',
        str(row.get('phone', '')) or '',
        str(row.get('state', '')) or '',
        str(row.get('chain', '')) or ''
    ]
    query = ' '.join([p for p in parts if p.strip()])
    encoded_query = urllib.parse.quote_plus(query)
    return f'https://www.bing.com/search?q={encoded_query}'

# Fix SettingWithCopyWarning by using .loc for assignment
filtered_df.loc[:, 'DuckDuckGo_Search_URL'] = filtered_df.apply(build_duckduckgo_search_url, axis=1)
filtered_df.loc[:, 'Bing_Search_URL'] = filtered_df.apply(build_bing_search_url, axis=1)

# Show a sample of all search URLs
filtered_df[['Google_Search_URL','DuckDuckGo_Search_URL','Bing_Search_URL']].head()

# Filter rows where Scraped_zipcode_to_label_match_rate contains '2/6' (not exact match)
filtered_2_6_df = filtered_df[filtered_df['Scraped_zipcode_to_label_match_rate'].astype(str).str.contains('4/6', na=False)]

# Show a sample of the filtered DataFrame
filtered_2_6_df.head()


C:\Users\clint\AppData\Local\Temp\ipykernel_21260\1280733105.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[:, 'DuckDuckGo_Search_URL'] = filtered_df.apply(build_duckduckgo_search_url, axis=1)
C:\Users\clint\AppData\Local\Temp\ipykernel_21260\1280733105.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[:, 'Bing_Search_URL'] = filtered_df.apply(build_bing_search_url, axis=1)


,clean_line1,clean_line2,line3,city,zip_code,label,phone,year,major_city,state,...,Label_scraped_matches_row_ids_parsed,Chain_scraped_matches_row_ids_parsed,Scraped_zipcode_to_label_match_rate,Scraped_phone_match_rate,Phone_Yelp_matches_row_ids,Yelp_phone_match_rate,place_identifier(year),Google_Search_URL,DuckDuckGo_Search_URL,Bing_Search_URL
804,"Wilmington , 90748 Wilmington Truck Stop Rd N )",310-830-3388 Alameda & Eubanks,D,Wilmington,90748,Wilmington Truck Stop Rd N ),310-830-3388,2006,Wilmington,CA,...,"[11, 12, 30, 34, 35, 36, 37, 38, 39, 40, 41, 4...",[],4/6 successful match,False,NaN,False,135,https://www.google.com/search?q=Wilmington+Tru...,https://duckduckgo.com/?q=Wilmington+Truck+Sto...,https://www.bing.com/search?q=Wilmington+Truck...
805,"Wilmington , 90748 Speedy Fuel",310-830-3388 Alameda & Eubanks,485 M,Wilmington,90748,Speedy Fuel,310-830-3388,2007,Wilmington,CA,...,"[86, 90, 91, 101, 116, 145, 153, 190, 261, 294...",[],4/6 successful match,False,NaN,False,135,https://www.google.com/search?q=Speedy+Fuel+Wi...,https://duckduckgo.com/?q=Speedy+Fuel+Wilmingt...,https://www.bing.com/search?q=Speedy+Fuel+Wilm...
806,"Wilmington , 90748 Speedy Fuel B ( Laval Rd W )",310-830-3388 Alameda & Eubanks,24 <U+2610><U+2610>,Wilmington,90748,Speedy Fuel B ( Laval Rd W ),310-830-3388,2008,Wilmington,CA,...,"[86, 90, 91, 101, 116, 145, 153, 190, 261, 294...",[],4/6 successful match,False,NaN,False,135,https://www.google.com/search?q=Speedy+Fuel+B+...,https://duckduckgo.com/?q=Speedy+Fuel+B+%28+La...,https://www.bing.com/search?q=Speedy+Fuel+B+%2...
807,D Wilmington ( 90748 ) Speedy Fuel # 7,310-830-3388 Alameda & Eubanks ( 1.5 mi S of C...,1 ) 24 22,Wilmington,90748,Speedy Fuel # 7,310-830-3388,2014,Wilmington,CA,...,"[86, 90, 91, 101, 116, 145, 153, 190, 261, 294...",[],4/6 successful match,False,NaN,False,135,https://www.google.com/search?q=Speedy+Fuel+%2...,https://duckduckgo.com/?q=Speedy+Fuel+%23+7+Wi...,https://www.bing.com/search?q=Speedy+Fuel+%23+...
808,Wilmington ( 90748 ) Speedy Fuel # 7 Rd W ),310-830-3388 Alameda &,) 24 <U+2610>,Wilmington,90748,Speedy Fuel # 7 Rd W ),310-830-3388,2015,Wilmington,CA,...,"[86, 90, 91, 101, 116, 145, 153, 190, 261, 294...",[],4/6 successful match,False,NaN,False,135,https://www.google.com/search?q=Speedy+Fuel+%2...,https://duckduckgo.com/?q=Speedy+Fuel+%23+7+Rd...,https://www.bing.com/search?q=Speedy+Fuel+%23+...


In [54]:
import webbrowser
import time
import pandas as pd

# Function to open all search URLs for a given DataFrame and column name, with progress output
def open_search_links(df, url_column, delay=1):
    urls = df[url_column].dropna()
    if urls.empty:
        print(f'No URLs found in column: {url_column}')
        return
    print(f'Opening {len(urls)} URLs from column: {url_column}...')
    for i, url in enumerate(urls, 1):
        print(f'Opening {i}/{len(urls)}: {url}')
        webbrowser.open(url)
        time.sleep(delay)  # Delay to avoid opening too many tabs at once
    print('Done opening URLs.')

# Function to open Google, DuckDuckGo, and Bing URLs in order for each row
def open_all_search_links_rowwise(df, delay=1):
    url_columns = ['Google_Search_URL', 'DuckDuckGo_Search_URL', 'Bing_Search_URL']
    for idx, row in df.iterrows():
        for col in url_columns:
            url = row.get(col)
            if pd.notna(url):
                print(f"Opening {col} for row {idx}: {url}")
                webbrowser.open(url)
                time.sleep(delay)
    print('Done opening all search URLs row by row.')

# Example usage: open all Google search links for filtered_2_6_df
## Uncomment ONE of the lines below to run (remove the leading #):
# open_search_links(filtered_2_6_df, 'Google_Search_URL')
# open_search_links(filtered_2_6_df, 'DuckDuckGo_Search_URL')
# open_search_links(filtered_2_6_df, 'Bing_Search_URL')

# Example usage: open all search links for filtered_2_6_df in order
# Uncomment the line below to run (remove the leading #):
open_all_search_links_rowwise(filtered_2_6_df, delay=1)

Opening Google_Search_URL for row 804: https://www.google.com/search?q=Wilmington+Truck+Stop+Rd+N+%29+Wilmington+90748+310-830-3388+CA+nan
Opening DuckDuckGo_Search_URL for row 804: https://duckduckgo.com/?q=Wilmington+Truck+Stop+Rd+N+%29+Wilmington+90748+310-830-3388+CA+nan
Opening DuckDuckGo_Search_URL for row 804: https://duckduckgo.com/?q=Wilmington+Truck+Stop+Rd+N+%29+Wilmington+90748+310-830-3388+CA+nan
Opening Bing_Search_URL for row 804: https://www.bing.com/search?q=Wilmington+Truck+Stop+Rd+N+%29+Wilmington+90748+310-830-3388+CA+nan
Opening Bing_Search_URL for row 804: https://www.bing.com/search?q=Wilmington+Truck+Stop+Rd+N+%29+Wilmington+90748+310-830-3388+CA+nan
Opening Google_Search_URL for row 805: https://www.google.com/search?q=Speedy+Fuel+Wilmington+90748+310-830-3388+CA+nan
Opening Google_Search_URL for row 805: https://www.google.com/search?q=Speedy+Fuel+Wilmington+90748+310-830-3388+CA+nan
Opening DuckDuckGo_Search_URL for row 805: https://duckduckgo.com/?q=Speedy+F